In [1]:
! unzip ./all.zip

Archive:  ./all.zip
  inflating: data/private_test_data.csv  
  inflating: data/question_meta.csv  
  inflating: data/student_meta.csv   
  inflating: data/subject_meta.csv   
  inflating: data/test_data.csv      
  inflating: data/train_data.csv     
  inflating: data/train_sparse.npz   
  inflating: data/valid_data.csv     
  inflating: majority_vote.py        
 extracting: part_a/__init__.py      
 extracting: part_a/ensemble.py      
  inflating: part_a/item_response.py  
  inflating: part_a/knn.py           
  inflating: part_a/matrix_factorization.py  
  inflating: part_a/neural_network.py  
  inflating: utils.py                


In [19]:
from utils import *
import random
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import copy
# import numpy as np
import torch

import random
import matplotlib.pyplot as plt

import autograd.numpy as np
from autograd import grad

In [41]:
def sigmoid(x):
    """ Apply sigmoid function.
    """
    return np.exp(x) / (1 + np.exp(x))


def neg_log_likelihood(data, theta, beta):
    """ Compute the negative log-likelihood.

    You may optionally replace the function arguments to receive a matrix.

    :param data: A dictionary {user_id: list, question_id: list,
    is_correct: list}
    :param theta: Vector
    :param beta: Vector
    :return: float
    """

    log_lklihood = 0.
    assert len(data["user_id"]) == len(data["question_id"])
    assert len(data["user_id"]) == len(data["is_correct"])
    length = len(data["user_id"])
    for s in range(length):
        i = data["user_id"][s]
        j = data["question_id"][s]
        cij = data["is_correct"][s]
        
        theta_i = theta[i]
        beta_j = beta[j]

        theta_i_minus_beta_j = theta_i - beta_j
        log_lklihood += cij*theta_i_minus_beta_j - np.log(1.0+np.exp(theta_i_minus_beta_j))
    return -log_lklihood

# def loss_auto_grad(data, c, theta, beta, k):
#   def theta


def update_theta_beta(data, lr, theta, beta):
    """ Update theta and beta using gradient descent.

    You are using alternating gradient descent. Your update should look:
    for i in iterations ...
        theta <- new_theta
        beta <- new_beta

    You may optionally replace the function arguments to receive a matrix.

    :param data: A dictionary {user_id: list, question_id: list,
    is_correct: list}
    :param lr: float
    :param theta: Vector
    :param beta: Vector
    :return: tuple of vectors
    """
    #####################################################################
    # TODO:                                                             #
    # Implement the function as described in the docstring.             #
    #####################################################################
    dtheta = []
    dbeta = []
    
    N = len(theta)
    M = len(beta)
    
    theta_mid = {x: [] for x in range(N)}
    beta_mid = {x: [] for x in range(M)}
    
    assert len(data["user_id"]) == len(data["question_id"])
    assert len(data["user_id"]) == len(data["is_correct"])
    
    length = len(data["user_id"])
    for s in range(length):
        i = data["user_id"][s]
        j = data["question_id"][s]
        cij = data["is_correct"][s]
        theta_i = theta[i]
        beta_j = beta[j]
        theta_mid[i].append(cij - sigmoid(theta_i-beta_j))
        beta_mid[j].append(-1*cij+sigmoid(theta_i-beta_j))
    
    for i in range(N):
        dtheta.append(-1*sum(theta_mid[i]))
    for j in range(M):
        dbeta.append(-1*sum(beta_mid[j]))

    theta -= lr * np.array(dtheta, dtype=np.float32)
    beta -= lr * np.array(dbeta, dtype=np.float32)

    #####################################################################
    #                       END OF YOUR CODE                            #
    #####################################################################
    return theta, beta


def irt(data, val_data, lr, iterations):
    """ Train IRT model.

    You may optionally replace the function arguments to receive a matrix.

    :param data: A dictionary {user_id: list, question_id: list,
    is_correct: list}
    :param val_data: A dictionary {user_id: list, question_id: list,
    is_correct: list}
    :param lr: float
    :param iterations: int
    :return: (theta, beta, val_acc_lst)
    """
    # TODO: Initialize theta and beta.
    theta = np.array([0 for i in range(542)], dtype=np.float32)
    beta = np.array([0 for j in range(1774)], dtype=np.float32)

    train_acc_lst = []
    val_acc_lst = []
    train_nlld = []
    val_nlld = []

    for i in range(iterations):
        train_neg_lld = neg_log_likelihood(data, theta=theta, beta=beta)
        val_neg_lld = neg_log_likelihood(val_data, theta=theta, beta=beta)
        train_nlld.append(train_neg_lld)
        val_nlld.append(val_neg_lld)

        val_score = evaluate(data=val_data, theta=theta, beta=beta)
        val_acc_lst.append(val_score)
        
        train_score = evaluate(data=data, theta=theta, beta=beta)
        train_acc_lst.append(train_score)

        print("Step: {} \n Train NLLK: {}, Train Score: {} \n Val NLLK: {}, Val Score: {}".format(i, train_neg_lld, train_score, val_neg_lld, val_score))
        # print("NLLK: {} \t Score: {}".format(neg_lld, score))
        theta, beta = update_theta_beta(data, lr, theta, beta)

    # TODO: You may change the return values to achieve what you want.
    return theta, beta, train_acc_lst, val_acc_lst, train_nlld, val_nlld


def evaluate(data, theta, beta):
    """ Evaluate the model given data and return the accuracy.
    :param data: A dictionary {user_id: list, question_id: list,
    is_correct: list}

    :param theta: Vector
    :param beta: Vector
    :return: float
    """
    pred = []
    for i, q in enumerate(data["question_id"]):
        u = data["user_id"][i]
        x = (theta[u] - beta[q]).sum()
        p_a = sigmoid(x)
        pred.append(p_a >= 0.5)
    return np.sum((data["is_correct"] == np.array(pred))) \
           / len(data["is_correct"])


In [4]:
def loss(data, c, theta, beta, k):
    log_lklihood = 0.
    assert len(data["user_id"]) == len(data["question_id"])
    assert len(data["user_id"]) == len(data["is_correct"])
    length = len(data["user_id"])
    for s in range(length):
        i = data["user_id"][s]
        j = data["question_id"][s]
        cij = data["is_correct"][s]
        
        theta_i = theta[i]
        beta_j = beta[j]
        k_j = k[j]

        useful_term = k_j*(theta_i - beta_j)
        term1 = cij*np.log(c+(1-c)*sigmoid(useful_term))
        term2 = (1-cij)*np.log(1-c-(1-c)*sigmoid(useful_term))
        new_term = term1+term2
        log_lklihood += new_term
    return -log_lklihood


In [5]:
def update_params(data, c, lr, theta, beta, k):
    dtheta = []
    dbeta = []
    dk = []
    
    N = len(theta)
    M = len(beta)
    
    theta_mid = {x: [] for x in range(N)}
    beta_mid = {x: [] for x in range(M)}
    k_mid = {x: [] for x in range(M)}
    
    assert len(data["user_id"]) == len(data["question_id"])
    assert len(data["user_id"]) == len(data["is_correct"])
    
    length = len(data["user_id"])
    for s in range(length):
        i = data["user_id"][s]
        j = data["question_id"][s]
        cij = data["is_correct"][s]
        theta_i = theta[i]
        beta_j = beta[j]
        k_j = k[j]
        useful_term = k_j*(theta_i - beta_j)

        theta_term1 = cij*(1-c)*sigmoid(useful_term)*(1-sigmoid(useful_term))*k_j/(c+(1-c)*sigmoid(useful_term))
        theta_term2 = (1-cij)*(-1*(1-c)*sigmoid(useful_term)*(1-sigmoid(useful_term))*k_j)/(1-c-(1-c)*sigmoid(useful_term))
        
        beta_term1 = cij*(1-c)*sigmoid(useful_term)*(1-sigmoid(useful_term))*(-1)*k_j/(c+(1-c)*sigmoid(useful_term))
        beta_term2 = (1-cij)*(-1)*(1-c)*sigmoid(useful_term)*(1-sigmoid(useful_term))*(-1)*k_j/(1-c-(1-c)*sigmoid(useful_term))

        k_term1 = cij*(1-c)*sigmoid(useful_term)*(1-sigmoid(useful_term))*(theta_i-beta_j)/(c+(1-c)*sigmoid(useful_term))
        k_term2 = (1-cij)*(-1)*(1-c)*sigmoid(useful_term)*(1-sigmoid(useful_term))*(theta_i-beta_j)/(1-c-(1-c)*sigmoid(useful_term))

        theta_mid[i].append(theta_term1+theta_term2)
        beta_mid[j].append(beta_term1+beta_term2)
        k_mid[j].append(k_term1+k_term2)
    
    for i in range(N):
        dtheta.append(-1*sum(theta_mid[i]))
    for j in range(M):
        dbeta.append(-1*sum(beta_mid[j]))
    for j in range(M):
        dk.append(-1*sum(k_mid[j]))

    theta -= lr * np.array(dtheta, dtype=np.float32)
    beta -= lr * np.array(dbeta, dtype=np.float32)
    k -= lr * np.array(dk, dtype=np.float32)
    #####################################################################
    #                       END OF YOUR CODE                            #
    #####################################################################
    return theta, beta, k


In [6]:
def new_eval(data, c, theta, beta, k):
    pred = []
    for i, q in enumerate(data["question_id"]):
        u = data["user_id"][i]
        x = (k[q]*(theta[u] - beta[q])).sum()
        p_a = sigmoid(x)*(1-c)+c
        pred.append(p_a >= 0.5)
    return np.sum((data["is_correct"] == np.array(pred))) \
           / len(data["is_correct"])

In [7]:
def predict_private_csv(data, c, theta, beta, k):
    for i, q in enumerate(data["question_id"]):
        u = data["user_id"][i]
        x = (k[q]*(theta[u] - beta[q])).sum()
        p_a = sigmoid(x)*(1-c)+c
        data["is_correct"].append(int(p_a >= 0.5))
    return data

In [8]:
def new_irt(data, val_data, c, lr, iterations, theta, beta, k):
    """ Train IRT model.

    You may optionally replace the function arguments to receive a matrix.

    :param data: A dictionary {user_id: list, question_id: list,
    is_correct: list}
    :param val_data: A dictionary {user_id: list, question_id: list,
    is_correct: list}
    :param lr: float
    :param iterations: int
    :return: (theta, beta, val_acc_lst)
    """
    # TODO: Initialize theta and beta.

    train_acc_lst = []
    val_acc_lst = []
    train_nlld = []
    val_nlld = []

    for i in range(iterations):
        train_neg_lld = loss(data, c, theta, beta, k)
        val_neg_lld = loss(val_data, c, theta, beta, k)
        train_nlld.append(train_neg_lld)
        val_nlld.append(val_neg_lld)

        val_score = new_eval(val_data, c, theta, beta, k)
        val_acc_lst.append(val_score)
        
        train_score = new_eval(data, c, theta, beta, k)
        train_acc_lst.append(train_score)

        print("Step: {} \n Train NLLK: {}, Train Score: {} \n Val NLLK: {}, Val Score: {}".format(i, train_neg_lld, train_score, val_neg_lld, val_score))
        # print("NLLK: {} \t Score: {}".format(neg_lld, score))
        theta, beta, k = update_params(data, c, lr, theta, beta, k)
        # print(theta, beta, k)

    # TODO: You may change the return values to achieve what you want.
    return theta, beta, k, train_acc_lst, val_acc_lst, train_nlld, val_nlld


In [21]:
def calc_correctness_percentage(data):
  length = len(data['question_id'])
  result = {i:{"total":0, "correct":0} for i in range(1774)}
  for s in range(length):
    if data["is_correct"][s] == 1:
      result[data["question_id"][s]]["correct"] += 1
    result[data["question_id"][s]]["total"] += 1
  ret = []
  for i in range(1774):
    if result[i]["total"] == 0:
      ret.append(1.)
    else:
      ret.append(result[i]["correct"]/result[i]["total"])
  return np.array(ret)

In [24]:
def load_question_meta(path):
    if not os.path.exists(path):
        raise Exception("The specified path {} does not exist.".format(path))
    
    # Return a dictionary whose keys are question_id and values are the category of that question
    data = {}
    with open(path, "r") as csv_file:
        reader = csv.reader(csv_file)
        for row in reader:
            try:
                question_id = int(row[0])
                cate = eval(row[1])
                if question_id in data:
                    print("GG")
                if question_id not in data:
                    data[question_id] = []
                data[question_id].extend(cate)
            except ValueError:
                pass
    return data


In [25]:
train_data = load_train_csv("./data")
# You may optionally use the sparse matrix.
sparse_matrix = load_train_sparse("./data")
# print(sparse_matrix.shape)
# print(sparse_matrix)
val_data = load_valid_csv("./data")
# print(len(val_data["user_id"]))
test_data = load_public_test_csv("./data")

question_meta = load_question_meta("./data/question_meta.csv")

In [215]:
private_test = load_private_test_csv("./data")

In [27]:
NUMBER = 1
ALGEBRA = 17
GEOMETRY = 39
STATISTICS = 68

number_questions = []
algebra_questions = []
geometry_questions = []
statistics_questions = []
other_questions = []

for question_id in question_meta:
  cate = question_meta[question_id]
  flag = False
  if NUMBER in cate:
    number_questions.append(question_id)
    flag = True
  if ALGEBRA in cate:
    algebra_questions.append(question_id)
    flag = True
  if GEOMETRY in cate:
    geometry_questions.append(question_id)
    flag = True
  if STATISTICS in cate:
    statistics_questions.append(question_id)
    flag = True
  if not flag:
    other_questions.append(question_id)

In [30]:
len(question_meta)

1774

In [31]:
len(other_questions)+len(number_questions)+len(algebra_questions)+len(statistics_questions)+len(geometry_questions)

1795

In [49]:
def split_data(data):
  number = {
      "user_id": [],
      "question_id": [],
      "is_correct": []
  }
  algebra = {
      "user_id": [],
      "question_id": [],
      "is_correct": []
  }
  geometry = {
      "user_id": [],
      "question_id": [],
      "is_correct": []
  }
  statistics = {
      "user_id": [],
      "question_id": [],
      "is_correct": []
  }
  other = {
      "user_id": [],
      "question_id": [],
      "is_correct": []
  }
  length = len(data["question_id"])
  for s in range(length):
    curr_question = data["question_id"][s]
    flag = False
    if curr_question in number_questions:
      number["user_id"].append(data["user_id"][s])
      number["question_id"].append(curr_question)
      number["is_correct"].append(data["is_correct"][s])
      flag = True
    if curr_question in geometry_questions:
      geometry["user_id"].append(data["user_id"][s])
      geometry["question_id"].append(curr_question)
      geometry["is_correct"].append(data["is_correct"][s])
      flag = True
    if curr_question in algebra_questions:
      algebra["user_id"].append(data["user_id"][s])
      algebra["question_id"].append(curr_question)
      algebra["is_correct"].append(data["is_correct"][s])
      flag = True
    if curr_question in statistics_questions:
      statistics["user_id"].append(data["user_id"][s])
      statistics["question_id"].append(curr_question)
      statistics["is_correct"].append(data["is_correct"][s])
      flag = True
    if not flag:
      # print(curr_question)
      other["user_id"].append(data["user_id"][s])
      other["question_id"].append(curr_question)
      other["is_correct"].append(data["is_correct"][s])
  return number, algebra, geometry, statistics, other

In [214]:
def test_cluster_model(data, c, theta, beta, k):
  length = len(data["user_id"])
  pred = []
  for i, q in enumerate(data["question_id"]):
    u = data["user_id"][i]
    if q in number_questions:
      curr_theta = theta["number"]
      curr_beta = beta["number"]
      curr_k = k["number"]
    elif q in algebra_questions:
      curr_theta = theta["algebra"]
      curr_beta = beta["algebra"]
      curr_k = k["algebra"]
    elif q in geometry_questions:
      curr_theta = theta["geometry"]
      curr_beta = beta["geometry"]
      curr_k = k["geometry"]
    elif q in statistics_questions:
      curr_theta = theta["statistics"]
      curr_beta = beta["statistics"]
      curr_k = k["statistics"]
    else:
      curr_theta = theta["other"]
      curr_beta = beta["other"]
      curr_k = k["other"]
    x = (curr_k[q]*(curr_theta[u]-curr_beta[q])).sum()
    p_a = sigmoid(x)*(1-c)+c
    pred.append(p_a >= 0.5)
  return np.sum(test_data["is_correct"] == np.array(pred)) \
        / len(test_data["is_correct"])

def private_test_cluster_model(data, c, theta, beta, k):
  length = len(data["user_id"])
  pred = []
  for i, q in enumerate(data["question_id"]):
    u = data["user_id"][i]
    if q in number_questions:
      curr_theta = theta["number"]
      curr_beta = beta["number"]
      curr_k = k["number"]
    elif q in algebra_questions:
      curr_theta = theta["algebra"]
      curr_beta = beta["algebra"]
      curr_k = k["algebra"]
    elif q in geometry_questions:
      curr_theta = theta["geometry"]
      curr_beta = beta["geometry"]
      curr_k = k["geometry"]
    elif q in statistics_questions:
      curr_theta = theta["statistics"]
      curr_beta = beta["statistics"]
      curr_k = k["statistics"]
    else:
      curr_theta = theta["other"]
      curr_beta = beta["other"]
      curr_k = k["other"]
    x = (curr_k[q]*(curr_theta[u]-curr_beta[q])).sum()
    p_a = sigmoid(x)*(1-c)+c
    data["is_correct"].append(int(p_a >= 0.5))
  return data

In [ ]:
private_test_cluster_model()

In [57]:
number_train, algebra_train, geometry_train, statistics_train, other_train = split_data(train_data)
number_valid, algebra_valid, geometry_valid, statistics_valid, other_valid = split_data(val_data)
number_test, algebra_test, geometry_test, statistics_test, other_test = split_data(test_data)

In [54]:
len(number_train["question_id"]),len(algebra_train["question_id"]),len(geometry_train["question_id"]),len(statistics_train["question_id"]),len(other_train["question_id"])

(25073, 12187, 14388, 5177, 508)

In [56]:
len(number_valid["question_id"]),len(algebra_valid["question_id"]),len(geometry_valid["question_id"]),len(statistics_valid["question_id"]),len(other_valid["question_id"])

(3089, 1539, 1818, 654, 58)

In [69]:
len(number_test["question_id"]),len(algebra_test["question_id"]),len(geometry_test["question_id"]),len(statistics_test["question_id"]),len(other_test["question_id"])

(1604, 760, 879, 318, 28)

In [162]:
def train_category(train, valid, iterations, lr, c):
  correctness = (0.1+calc_correctness_percentage(train))*10
  init_theta = np.array([0 for i in range(542)], dtype=np.float32)
  init_beta = np.array([0 for j in range(1774)], dtype=np.float32)
  init_k = correctness
  return new_irt(train, valid, c, lr, iterations, init_theta, init_beta, init_k)

In [146]:
alg_theta, alg_beta, alg_k, train_acc_lst, val_acc_lst, train_nlld, val_nlld = train_category(algebra_train, algebra_valid, 20, 0.0015, 0.25)

Step: 0 
 Train NLLK: 8128.814661318239, Train Score: 0.6143431525395914 
 Val NLLK: 1024.7227034311363, Val Score: 0.6166341780376868
Step: 1 
 Train NLLK: 6984.674517329807, Train Score: 0.6686633297776319 
 Val NLLK: 916.4097632290305, Val Score: 0.6640675763482781
Step: 2 
 Train NLLK: 6697.569126303753, Train Score: 0.6959875276934439 
 Val NLLK: 894.6645898170179, Val Score: 0.682261208576998
Step: 3 
 Train NLLK: 6540.239408606103, Train Score: 0.7125625666694019 
 Val NLLK: 886.008896722261, Val Score: 0.6796621182586094
Step: 4 
 Train NLLK: 6436.776887813398, Train Score: 0.7235578895544432 
 Val NLLK: 881.1164350405439, Val Score: 0.6855100714749838
Step: 5 
 Train NLLK: 6362.294254183627, Train Score: 0.7331582834167556 
 Val NLLK: 878.4844444492484, Val Score: 0.6894087069525666
Step: 6 
 Train NLLK: 6305.626256739092, Train Score: 0.7396406006400262 
 Val NLLK: 877.0320315020722, Val Score: 0.6939571150097466
Step: 7 
 Train NLLK: 6260.85032101809, Train Score: 0.74210224

In [147]:
new_eval(algebra_test, C, alg_theta, alg_beta, alg_k)

0.718421052631579

In [150]:
num_theta, num_beta, num_k, train_acc_lst, val_acc_lst, train_nlld, val_nlld = train_category(number_train, number_valid, 20, 0.002, 0.25)

Step: 0 
 Train NLLK: 16988.18162538154, Train Score: 0.5937063773780561 
 Val NLLK: 2090.884066071254, Val Score: 0.5950145678213014
Step: 1 
 Train NLLK: 14650.977050556268, Train Score: 0.6723966019223866 
 Val NLLK: 1818.0400683085538, Val Score: 0.6681774036905147
Step: 2 
 Train NLLK: 14108.373145436564, Train Score: 0.6896262912296095 
 Val NLLK: 1766.365130259585, Val Score: 0.691485917772742
Step: 3 
 Train NLLK: 13837.59131466964, Train Score: 0.7042236668926734 
 Val NLLK: 1745.1324749614162, Val Score: 0.7028164454516025
Step: 4 
 Train NLLK: 13662.610575647177, Train Score: 0.7152714074901289 
 Val NLLK: 1729.9015644321382, Val Score: 0.7118808675946908
Step: 5 
 Train NLLK: 13540.688686456886, Train Score: 0.7196585968970606 
 Val NLLK: 1722.3814978380956, Val Score: 0.716413078666235
Step: 6 
 Train NLLK: 13451.878988114835, Train Score: 0.7243249710844335 
 Val NLLK: 1716.2327240465124, Val Score: 0.7202978310132729
Step: 7 
 Train NLLK: 13384.040375597415, Train Score:

In [151]:
new_eval(number_test, C, num_theta, num_beta, num_k)

0.7182044887780549

In [165]:
sta_theta, sta_beta, sta_k, train_acc_lst, val_acc_lst, train_nlld, val_nlld = train_category(statistics_train, statistics_valid, 23, 0.0015, 0.25)

Step: 0 
 Train NLLK: 3457.4141642556087, Train Score: 0.6127100637434808 
 Val NLLK: 436.1104307157434, Val Score: 0.6146788990825688
Step: 1 
 Train NLLK: 3131.5117859660427, Train Score: 0.6260382460884683 
 Val NLLK: 407.3585690703952, Val Score: 0.6253822629969419
Step: 2 
 Train NLLK: 2989.462995759414, Train Score: 0.6581031485416264 
 Val NLLK: 395.3519927007386, Val Score: 0.6498470948012233
Step: 3 
 Train NLLK: 2902.820168596804, Train Score: 0.6870774579872513 
 Val NLLK: 388.5613085792711, Val Score: 0.6590214067278287
Step: 4 
 Train NLLK: 2842.479203987441, Train Score: 0.7040757195286845 
 Val NLLK: 384.18918215945524, Val Score: 0.6712538226299695
Step: 5 
 Train NLLK: 2797.299982628916, Train Score: 0.7212671431330887 
 Val NLLK: 381.14084538277564, Val Score: 0.6880733944954128
Step: 6 
 Train NLLK: 2761.8513234627935, Train Score: 0.7282209774000387 
 Val NLLK: 378.89806153494385, Val Score: 0.6865443425076453
Step: 7 
 Train NLLK: 2733.1003686352847, Train Score: 0

In [166]:
new_eval(statistics_test, C, sta_theta, sta_beta, sta_k)

0.6918238993710691

In [163]:
geo_theta, geo_beta, geo_k, train_acc_lst, val_acc_lst, train_nlld, val_nlld = train_category(geometry_train, geometry_valid, 12, 0.0015, 0.25)

Step: 0 
 Train NLLK: 9781.902479668563, Train Score: 0.5891715318320823 
 Val NLLK: 1234.010036426877, Val Score: 0.5913091309130913
Step: 1 
 Train NLLK: 8494.98894684706, Train Score: 0.6501251042535446 
 Val NLLK: 1112.5942462527119, Val Score: 0.6507150715071507
Step: 2 
 Train NLLK: 8154.341437209967, Train Score: 0.6768835140394773 
 Val NLLK: 1086.1154398928866, Val Score: 0.676017601760176
Step: 3 
 Train NLLK: 7967.715467041529, Train Score: 0.6959966638865721 
 Val NLLK: 1073.5006537714019, Val Score: 0.6826182618261826
Step: 4 
 Train NLLK: 7846.255194721766, Train Score: 0.7128857381150959 
 Val NLLK: 1066.5378188187308, Val Score: 0.6837183718371838
Step: 5 
 Train NLLK: 7759.950853071328, Train Score: 0.7211565193216569 
 Val NLLK: 1062.5291056658193, Val Score: 0.6859185918591859
Step: 6 
 Train NLLK: 7695.135391322452, Train Score: 0.7286627745343341 
 Val NLLK: 1060.1873377631293, Val Score: 0.693069306930693
Step: 7 
 Train NLLK: 7644.54765084595, Train Score: 0.7329

In [164]:
new_eval(geometry_test, C, geo_theta, geo_beta, geo_k)

0.6962457337883959

In [187]:
other_theta, other_beta, other_k, train_acc_lst, val_acc_lst, train_nlld, val_nlld = train_category(other_train, other_valid, 20, 0.0015, 0.25)

Step: 0 
 Train NLLK: 352.67595775665063, Train Score: 0.5610236220472441 
 Val NLLK: 40.54167671416844, Val Score: 0.5517241379310345
Step: 1 
 Train NLLK: 335.38685103706257, Train Score: 0.5610236220472441 
 Val NLLK: 39.011171577731766, Val Score: 0.5517241379310345
Step: 2 
 Train NLLK: 325.62649297154195, Train Score: 0.5610236220472441 
 Val NLLK: 38.41588366534773, Val Score: 0.5517241379310345
Step: 3 
 Train NLLK: 318.0770137102112, Train Score: 0.5866141732283464 
 Val NLLK: 38.09614847126977, Val Score: 0.5517241379310345
Step: 4 
 Train NLLK: 311.717439083919, Train Score: 0.6240157480314961 
 Val NLLK: 37.8948125494497, Val Score: 0.5689655172413793
Step: 5 
 Train NLLK: 306.13634350678325, Train Score: 0.6633858267716536 
 Val NLLK: 37.755100186727745, Val Score: 0.603448275862069
Step: 6 
 Train NLLK: 301.1179792953836, Train Score: 0.6850393700787402 
 Val NLLK: 37.651100414767285, Val Score: 0.5862068965517241
Step: 7 
 Train NLLK: 296.53576025864504, Train Score: 0.7

In [171]:
all_theta, all_beta, all_k, train_acc_lst, val_acc_lst, train_nlld, val_nlld = train_category(train_data, val_data, 20, 0.001, 0.25)

Step: 0 
 Train NLLK: 38272.00023409504, Train Score: 0.5984335309060118 
 Val NLLK: 4775.5714064688855, Val Score: 0.6007620660457239
Step: 1 
 Train NLLK: 33773.36175206723, Train Score: 0.6742520462884561 
 Val NLLK: 4267.8889145779785, Val Score: 0.6785210273779283
Step: 2 
 Train NLLK: 32423.58326478415, Train Score: 0.6793677674287327 
 Val NLLK: 4114.19515671614, Val Score: 0.6800733841377364
Step: 3 
 Train NLLK: 32047.78715351883, Train Score: 0.6957027942421676 
 Val NLLK: 4115.133272642183, Val Score: 0.6927744848998024
Step: 4 
 Train NLLK: 31724.924983496818, Train Score: 0.6936212249506069 
 Val NLLK: 4068.0328114375966, Val Score: 0.6840248377081569
Step: 5 
 Train NLLK: 31327.959438681828, Train Score: 0.7082274908269828 
 Val NLLK: 4054.2077687008796, Val Score: 0.7011007620660458
Step: 6 
 Train NLLK: 31001.639253037705, Train Score: 0.7130080440304827 
 Val NLLK: 4016.782127808603, Val Score: 0.6974315551792266
Step: 7 
 Train NLLK: 30882.26270483196, Train Score: 0.

In [190]:
new_eval(statistics_test, C, sta_theta, sta_beta, sta_k)

0.6918238993710691

In [213]:
new_eval(other_valid, C, geo_theta, geo_beta, geo_k)

0.6428571428571429

In [221]:
theta = {
    "number": num_theta,
    "algebra": alg_theta,
    "geometry": all_theta,
    "statistics": all_theta,
    "other": all_theta
}
beta = {
    "number": num_beta,
    "algebra": alg_beta,
    "geometry": all_beta,
    "statistics": all_beta,
    "other": all_beta
}
k = {
    "number": num_k,
    "algebra": alg_k,
    "geometry": all_k,
    "statistics": all_k,
    "other": all_k
}

test_cluster_model(test_data, 0.25, theta, beta, k)

0.7143663561953147

In [222]:
result = private_test_cluster_model(private_test, 0.25, theta, beta, k)
save_private_test_csv(result)

In [134]:
correctness = (0.1+calc_correctness_percentage(_data))*10
init_theta = np.array([0 for i in range(542)], dtype=np.float32)
init_beta = np.array([0 for j in range(1774)], dtype=np.float32)
init_k = correctness

NUM_ITERS = 20
LEARNING_RATE = 0.0015
C = 0.25
alg_theta, alg_beta, alg_k, train_acc_lst, val_acc_lst, train_nlld, val_nlld = new_irt(algebra_train, algebra_valid, C, LEARNING_RATE, NUM_ITERS, init_theta, init_beta, init_k)

Step: 0 
 Train NLLK: 8128.814661318239, Train Score: 0.6143431525395914 
 Val NLLK: 1024.7227034311363, Val Score: 0.6166341780376868
Step: 1 
 Train NLLK: 6984.674517329807, Train Score: 0.6686633297776319 
 Val NLLK: 916.4097632290305, Val Score: 0.6640675763482781
Step: 2 
 Train NLLK: 6697.569126303753, Train Score: 0.6959875276934439 
 Val NLLK: 894.6645898170179, Val Score: 0.682261208576998
Step: 3 
 Train NLLK: 6540.239408606103, Train Score: 0.7125625666694019 
 Val NLLK: 886.008896722261, Val Score: 0.6796621182586094
Step: 4 
 Train NLLK: 6436.776887813398, Train Score: 0.7235578895544432 
 Val NLLK: 881.1164350405439, Val Score: 0.6855100714749838
Step: 5 
 Train NLLK: 6362.294254183627, Train Score: 0.7331582834167556 
 Val NLLK: 878.4844444492484, Val Score: 0.6894087069525666
Step: 6 
 Train NLLK: 6305.626256739092, Train Score: 0.7396406006400262 
 Val NLLK: 877.0320315020722, Val Score: 0.6939571150097466
Step: 7 
 Train NLLK: 6260.85032101809, Train Score: 0.74210224

In [16]:
theta, beta, k, train_acc_lst, val_acc_lst, train_nlld, val_nlld = new_irt(train_data, val_data, C, 0.0009, 5, theta, beta, k)

In [135]:
new_eval(algebra_test, C, theta, beta, k)

0.718421052631579

In [223]:
length = len(other_train["is_correct"])
data_points = []
for i in range(length):
  data_points.append((other_train["user_id"][i], other_train["question_id"][i], other_train["is_correct"][i]))

In [224]:
sample1 = random.choices(data_points, k=len(data_points))
sample2 = random.choices(data_points, k=len(data_points))
sample3 = random.choices(data_points, k=len(data_points))
sample4 = random.choices(data_points, k=len(data_points))
sample5 = random.choices(data_points, k=len(data_points))

In [92]:
def datapoint2dict(sample):
  length = len(sample)
  result = {"user_id":[], "question_id":[], "is_correct":[]}
  for i in range(length):
    result["user_id"].append(sample[i][0])
    result["question_id"].append(sample[i][1])
    result["is_correct"].append(sample[i][2])
  return result

In [225]:
sample1 = datapoint2dict(sample1)
sample2 = datapoint2dict(sample2)
sample3 = datapoint2dict(sample3)
sample4 = datapoint2dict(sample4)
sample5 = datapoint2dict(sample5)

In [234]:
correctness = (0.1+calc_correctness_percentage(sample1))*5
init_theta = np.array([0 for i in range(542)], dtype=np.float32)
init_beta = np.array([0 for j in range(1774)], dtype=np.float32)
init_k = correctness

NUM_ITERS = 37
LEARNING_RATE = 0.005
C = 0.25
theta1, beta1, k1, train_acc_lst, val_acc_lst, train_nlld, val_nlld = new_irt(sample1, other_valid, C, LEARNING_RATE, NUM_ITERS, init_theta, init_beta, init_k)

Step: 0 
 Train NLLK: 358.80586524184287, Train Score: 0.5374015748031497 
 Val NLLK: 40.54167671416844, Val Score: 0.5517241379310345
Step: 1 
 Train NLLK: 340.02819740911445, Train Score: 0.5374015748031497 
 Val NLLK: 39.344028409433264, Val Score: 0.5517241379310345
Step: 2 
 Train NLLK: 326.77815027883423, Train Score: 0.5393700787401575 
 Val NLLK: 38.64098970168451, Val Score: 0.5517241379310345
Step: 3 
 Train NLLK: 316.5609109600379, Train Score: 0.5708661417322834 
 Val NLLK: 38.19915521471594, Val Score: 0.5517241379310345
Step: 4 
 Train NLLK: 308.16836590422935, Train Score: 0.6614173228346457 
 Val NLLK: 37.89742270071134, Val Score: 0.5689655172413793
Step: 5 
 Train NLLK: 300.9888399153471, Train Score: 0.687007874015748 
 Val NLLK: 37.676938418689176, Val Score: 0.6206896551724138
Step: 6 
 Train NLLK: 294.6758463492927, Train Score: 0.7106299212598425 
 Val NLLK: 37.507165903730865, Val Score: 0.6379310344827587
Step: 7 
 Train NLLK: 289.0159480386457, Train Score: 0.

In [239]:
correctness = (0.1+calc_correctness_percentage(sample2))*5
init_theta = np.array([0 for i in range(542)], dtype=np.float32)
init_beta = np.array([0 for j in range(1774)], dtype=np.float32)
init_k = correctness

NUM_ITERS = 37
LEARNING_RATE = 0.0025
C = 0.25
theta2, beta2, k2, train_acc_lst, val_acc_lst, train_nlld, val_nlld = new_irt(sample2, other_valid, C, LEARNING_RATE, NUM_ITERS, init_theta, init_beta, init_k)

Step: 0 
 Train NLLK: 352.16513213288476, Train Score: 0.562992125984252 
 Val NLLK: 40.54167671416844, Val Score: 0.5517241379310345
Step: 1 
 Train NLLK: 340.3034463055558, Train Score: 0.562992125984252 
 Val NLLK: 39.9750459250709, Val Score: 0.5517241379310345
Step: 2 
 Train NLLK: 330.79247825017444, Train Score: 0.562992125984252 
 Val NLLK: 39.61761147325051, Val Score: 0.5517241379310345
Step: 3 
 Train NLLK: 322.9070512240169, Train Score: 0.562992125984252 
 Val NLLK: 39.39696094318149, Val Score: 0.5517241379310345
Step: 4 
 Train NLLK: 316.1432369635792, Train Score: 0.5728346456692913 
 Val NLLK: 39.25983450495286, Val Score: 0.5517241379310345
Step: 5 
 Train NLLK: 310.19093604572976, Train Score: 0.5748031496062992 
 Val NLLK: 39.173264522982315, Val Score: 0.5517241379310345
Step: 6 
 Train NLLK: 304.85605906091746, Train Score: 0.594488188976378 
 Val NLLK: 39.11764354986249, Val Score: 0.5689655172413793
Step: 7 
 Train NLLK: 300.0103027052915, Train Score: 0.6122047

In [247]:
correctness = (0.1+calc_correctness_percentage(sample3))*5
init_theta = np.array([0 for i in range(542)], dtype=np.float32)
init_beta = np.array([0 for j in range(1774)], dtype=np.float32)
init_k = correctness

NUM_ITERS = 37
LEARNING_RATE = 0.001
C = 0.25
theta3, beta3, k3, train_acc_lst, val_acc_lst, train_nlld, val_nlld = new_irt(sample3, other_valid, C, LEARNING_RATE, NUM_ITERS, init_theta, init_beta, init_k)

Step: 0 
 Train NLLK: 348.0785271427566, Train Score: 0.5787401574803149 
 Val NLLK: 40.54167671416844, Val Score: 0.5517241379310345
Step: 1 
 Train NLLK: 344.0519844531861, Train Score: 0.5787401574803149 
 Val NLLK: 40.27984921077136, Val Score: 0.5517241379310345
Step: 2 
 Train NLLK: 340.4213113162512, Train Score: 0.5787401574803149 
 Val NLLK: 40.056519065803066, Val Score: 0.5517241379310345
Step: 3 
 Train NLLK: 337.13026816709976, Train Score: 0.5787401574803149 
 Val NLLK: 39.86704732485355, Val Score: 0.5517241379310345
Step: 4 
 Train NLLK: 334.12297793697235, Train Score: 0.5787401574803149 
 Val NLLK: 39.70650786964199, Val Score: 0.5517241379310345
Step: 5 
 Train NLLK: 331.35006722521956, Train Score: 0.5787401574803149 
 Val NLLK: 39.570299209609026, Val Score: 0.5517241379310345
Step: 6 
 Train NLLK: 328.77055646116975, Train Score: 0.5787401574803149 
 Val NLLK: 39.45440742225175, Val Score: 0.5517241379310345
Step: 7 
 Train NLLK: 326.35151964388314, Train Score: 0

In [126]:
def evaluate_ensemble(data, c, theta_lst, beta_lst, k_lst):
    pred = []
    for i, q in enumerate(data["question_id"]):
        u = data["user_id"][i]
        pas = []
        for j in range(len(theta_lst)):
          x = (k_lst[j][q]*(theta_lst[j][u] - beta_lst[j][q])).sum()
          p_a = sigmoid(x)*(1-c)+c
          pas.append(p_a)
        pred.append(sum(pas)/len(pas) >= 0.5)
    return np.sum((data["is_correct"] == np.array(pred))) \
           / len(data["is_correct"])

In [250]:
correctness = (0.1+calc_correctness_percentage(sample4))*5
init_theta = np.array([0 for i in range(542)], dtype=np.float32)
init_beta = np.array([0 for j in range(1774)], dtype=np.float32)
init_k = correctness

NUM_ITERS = 40
LEARNING_RATE = 0.0025
C = 0.25
theta4, beta4, k4, train_acc_lst, val_acc_lst, train_nlld, val_nlld = new_irt(sample4, other_valid, C, LEARNING_RATE, NUM_ITERS, init_theta, init_beta, init_k)

Step: 0 
 Train NLLK: 337.86201466743665, Train Score: 0.6181102362204725 
 Val NLLK: 40.54167671416844, Val Score: 0.5517241379310345
Step: 1 
 Train NLLK: 328.03832559159423, Train Score: 0.6181102362204725 
 Val NLLK: 40.06691495874893, Val Score: 0.5517241379310345
Step: 2 
 Train NLLK: 319.97182642669566, Train Score: 0.6181102362204725 
 Val NLLK: 39.741281771118025, Val Score: 0.5517241379310345
Step: 3 
 Train NLLK: 313.1443775356285, Train Score: 0.6181102362204725 
 Val NLLK: 39.51929645367334, Val Score: 0.5517241379310345
Step: 4 
 Train NLLK: 307.1820512861802, Train Score: 0.6181102362204725 
 Val NLLK: 39.365182171212474, Val Score: 0.5517241379310345
Step: 5 
 Train NLLK: 301.8465242342443, Train Score: 0.6181102362204725 
 Val NLLK: 39.25520940704719, Val Score: 0.5517241379310345
Step: 6 
 Train NLLK: 296.98673391952553, Train Score: 0.639763779527559 
 Val NLLK: 39.174346633029884, Val Score: 0.5517241379310345
Step: 7 
 Train NLLK: 292.50358854394955, Train Score: 0

In [253]:
correctness = (0.1+calc_correctness_percentage(sample5))*5
init_theta = np.array([0 for i in range(542)], dtype=np.float32)
init_beta = np.array([0 for j in range(1774)], dtype=np.float32)
init_k = correctness

NUM_ITERS = 40
LEARNING_RATE = 0.003
C = 0.25
theta5, beta5, k5, train_acc_lst, val_acc_lst, train_nlld, val_nlld = new_irt(sample5, other_valid, C, LEARNING_RATE, NUM_ITERS, init_theta, init_beta, init_k)

Step: 0 
 Train NLLK: 351.1434808853528, Train Score: 0.5669291338582677 
 Val NLLK: 40.54167671416844, Val Score: 0.5517241379310345
Step: 1 
 Train NLLK: 337.4526062851103, Train Score: 0.5669291338582677 
 Val NLLK: 39.65473377739947, Val Score: 0.5517241379310345
Step: 2 
 Train NLLK: 327.66950098403515, Train Score: 0.5669291338582677 
 Val NLLK: 39.11003703653698, Val Score: 0.5517241379310345
Step: 3 
 Train NLLK: 319.9066283809888, Train Score: 0.5669291338582677 
 Val NLLK: 38.746008838827855, Val Score: 0.5517241379310345
Step: 4 
 Train NLLK: 313.2751378360813, Train Score: 0.5669291338582677 
 Val NLLK: 38.47658003321679, Val Score: 0.5517241379310345
Step: 5 
 Train NLLK: 307.38551804780104, Train Score: 0.5787401574803149 
 Val NLLK: 38.26329116117778, Val Score: 0.5517241379310345
Step: 6 
 Train NLLK: 302.0365386088658, Train Score: 0.6082677165354331 
 Val NLLK: 38.08735524258884, Val Score: 0.5517241379310345
Step: 7 
 Train NLLK: 297.1089046863402, Train Score: 0.633

In [255]:
evaluate_ensemble(other_test, 0.25, [theta1, theta2, theta3,theta4,theta5], [beta1, beta2, beta3,beta4,beta5], [k1,k2,k3,k4,k5])

0.5357142857142857

In [169]:
def predict_ensemble_private(data, c, theta_lst, beta_lst, k_lst):
    pred = []
    data = data.copy()
    for i, q in enumerate(data["question_id"]):
        u = data["user_id"][i]
        pas = []
        for j in range(len(theta_lst)):
          x = (k_lst[j][q]*(theta_lst[j][u] - beta_lst[j][q])).sum()
          p_a = sigmoid(x)*(1-c)+c
          pas.append(p_a >= 0.5)
        pred.append(int(sum(pas)/len(pas) >= 0.5))
    data["is_correct"] = pred
    return data

In [167]:
private_data = load_private_test_csv("./data")

In [174]:
result = predict_ensemble_private(private_data, 0.25, [theta1, theta2, theta3,theta4,theta5], [beta1, beta2, beta3,beta4,beta5], [k1,k2,k3,k4,k5])

In [176]:
save_private_test_csv(result)